In [82]:
import sys
import string
import os
import numpy as np
from math import pi
import scipy.stats
import warnings
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm import tqdm

from bokeh.layouts import gridplot
from bokeh.plotting import output_file, figure, show
from bokeh.palettes import inferno
from bokeh.models import Label
from bokeh.models import Range1d
from bokeh.io import export_png
%matplotlib inline
warnings.filterwarnings('ignore')

In [83]:
fig = plt.figure()
mpl.rcParams['figure.figsize'] = (10.0, 6.0)

In [84]:
print('Write the length of DNA in bp:')
bp = int(input().strip())

Write the length of DNA in bp:
10000


In [85]:
step = 1
fname = f'../DNA/dsDNA/{bp}bp/50mM/push_from_30nm/dsDNA_{bp}bp_mM50.0_pressure_push_from_30nm.dat'
with open(fname, 'r') as fh:
    max_frame = int(fh.readlines()[-1].split()[0]) // 10000
rads = np.array([float(i.split('\t')[1].strip()) for i in open(fname)])
rads = rads[2:]
rads

array([ 29.995001,  29.99    ,  29.985001, ...,  14.575   ,  14.57    ,
        14.565   ])

In [86]:
#Считываем посчитанное радиальноe распределение
if not os.path.exists(f'../DNA/dsDNA/{bp}bp/50mM/plots'):
        os.mkdir(f'../DNA/dsDNA/{bp}bp/50mM/plots')
        os.mkdir(f'../DNA/dsDNA/{bp}bp/50mM/plots/movie')
elems = ['DNA','Na','Cl']
x = np.arange(10,600)
for frame in tqdm(range(143, max_frame, 1)):
    #Параметры для графика BOKEH
    p = figure(x_axis_label='Distance, nm', y_axis_label=r'Density ⋅1e-8', 
            plot_width=800, plot_height=600,
           x_range = (0, 30), y_range = (0,15))
    p.xaxis.axis_label_text_font_size = "22pt"
    p.xaxis.major_label_text_font_size = "18pt"
    p.yaxis.axis_label_text_font_size = "22pt"
    p.yaxis.major_label_text_font_size = "18pt"
    colors = inferno(3+1)
    for c, elem in zip(colors,elems):
        fname = f'../DNA/dsDNA/{bp}bp/50mM/{elem}_rad_distr/{frame}.dat'
        dens = np.array([float(i.split('\t')[1].strip()) for i in open(fname)])
        p.line(x/10, dens/1e-8, color = c, legend=elem, line_width = 5)
    p.legend.label_text_font_size = "18pt"
    export_png(p, filename=f'../DNA/dsDNA/{bp}bp/50mM/plots/movie/{frame}.png')
    

 16%|█▌        | 457/2944 [11:28<1:02:24,  1.51s/it]

URLError: <urlopen error [Errno 111] Connection refused>

In [26]:
fname = f'../DNA_packaging/dsDNA/mM50_length/10000bp/push_from_30nm/dsDNA_10000bp_mM50.0_pressure_push_from_30nm.dat'
with open(fname, 'r') as fh:
    max_frame = int(fh.readlines()[-1].split()[0]) // 10000

elem = 'DNA'

if not os.path.exists(f'dsDNA_10000bp_mM50/plots_new'):
        os.mkdir(f'dsDNA_10000bp_mM50/plots_new')
# x = np.arange(10,600)
fig = plt.figure(figsize=(15,10))
for frame in tqdm(range(0, max_frame, 20)):
    fname = f'dsDNA_10000bp_mM50/{elem}_rad_distr_new/{frame}.dat'
    dens = np.array([float(i.split('\t')[1].strip()) for i in open(fname)])
    x = np.array([float(i.split('\t')[0].strip()) for i in open(fname)])
    plt.plot(x/10., dens, linewidth = 2)
#     plt.ylim(0,1.5e-7)
    plt.xlim(0,60)
    plt.savefig(f'dsDNA_10000bp_mM50/plots_new/{frame}.png')
    plt.clf()



100%|██████████| 155/155 [00:28<00:00,  5.43it/s]


In [168]:
fname = f'../DNA/dsDNA/{bp}bp/50mM/layers_test.dat'
lay_num =  np.array([float(i.split('\t')[1].strip()) for i in open(fname)])
lay_dist = np.array([float(i.split('\t')[2].strip()) for i in open(fname)])

In [169]:
p = figure(toolbar_location = None,
           x_axis_label='Radius of compression, nm', y_axis_label='Number of layers', 
           plot_width=800, plot_height=600)
p.xaxis.axis_label_text_font_size = "22pt"
p.xaxis.major_label_text_font_size = "18pt"
p.yaxis.axis_label_text_font_size = "22pt"
p.yaxis.major_label_text_font_size = "18pt"
c = inferno(3+1)[2]
p.scatter(rads, lay_num, color = c, line_color=None, fill_alpha=0.3, size=5)
p.legend.label_text_font_size = "18pt"
export_png(p, filename=f'../DNA/dsDNA/{bp}bp/50mM/plots/lay_num.png')
# show(p)

'/home/yagafarova/DNA/dsDNA/10000bp/50mM/plots/lay_num.png'

In [167]:
from sklearn import linear_model
x_train = (np.array([rads[::2]])).T
x_test = (np.array([rads[1::2]])).T

# Split the targets into training/testing sets
y_train = lay_dist[::2]
y_test = lay_dist[1::2]

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(x_train, y_train)

# Make predictions using the testing set

y_pred = regr.predict(x_test)

err = mean_squared_error(y_test, y_pred)
var_score = r2_score(y_test, y_pred)

leg = "Mean squared error: %.2f\t" % err + 'Variance score: %.2f' % var_score

p = figure(toolbar_location = None,
           x_axis_label='Radius of compression, nm', y_axis_label='Interlayer distance, nm', 
           plot_width=800, plot_height=600)
p.xaxis.axis_label_text_font_size = "22pt"
p.xaxis.major_label_text_font_size = "18pt"
p.yaxis.axis_label_text_font_size = "22pt"
p.yaxis.major_label_text_font_size = "18pt"
colorst = inferno(3+1)
mytext = Label(x=70, y=70, text=leg)

p.scatter(rads, lay_dist, line_color=None, fill_alpha=0.3, size=5)
p.line(rads[1::2], y_pred, color = colors[0], line_width = 3)
p.add_layout(mytext)

p.legend.label_text_font_size = "18pt"
export_png(p, filename=f'../DNA/dsDNA/{bp}bp/50mM/plots/lay_dist.png')

# The coefficients
# print('Coefficients: \n', regr.coef_)
# The mean squared error


# Explained variance score: 1 is perfect prediction
# print('Variance score: %.2f' % r2_score(y_test, y_pred))

'/home/yagafarova/DNA/dsDNA/10000bp/50mM/plots/lay_dist.png'